In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%%bash
ls

CleanData.ipynb
ClimbingLogBook.zip
Data Analysis.ipynb
WebScrappingTool.ipynb
database.sqlite


In [3]:
# Read sqlite query results into a pandas DataFrame
def qry(q, connection = sqlite3.connect("../PassionProject/database.sqlite")):
    df = pd.read_sql_query(q, connection)
    connection.close
    return df

In [4]:
tables = qry("""
    SELECT name FROM sqlite_master
""")

# Get Col + info
columns_in_table = pd.DataFrame()
for i in tables.name:
    df_i = qry("PRAGMA TABLE_INFO(" + i + ");")
    df_i['table_name'] = i
    columns_in_table = columns_in_table.append(df_i)
tables = tables.name
print(tables)

0      user
1    method
2     grade
3    ascent
Name: name, dtype: object


/var/folders/b8/qw7xf4r16w383xrhw3980l4w0000gp/T/ipykernel_11303/2482935748.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  columns_in_table = columns_in_table.append(df_i)
/var/folders/b8/qw7xf4r16w383xrhw3980l4w0000gp/T/ipykernel_11303/2482935748.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  columns_in_table = columns_in_table.append(df_i)
/var/folders/b8/qw7xf4r16w383xrhw3980l4w0000gp/T/ipykernel_11303/2482935748.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  columns_in_table = columns_in_table.append(df_i)
/var/folders/b8/qw7xf4r16w383xrhw3980l4w0000gp/T/ipykernel_11303/2482935748.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [5]:
# work w dataframes
df_user = qry("SELECT * FROM USER")
df_grade = qry("SELECT * FROM grade")
df_method = qry("SELECT * FROM method")
df_ascent = qry("SELECT * FROM ascent")

In [6]:
df_user

,id,first_name,last_name,city,country,sex,height,weight,started,competitions,...,sponsor2,sponsor3,best_area,worst_area,guide_area,interests,birth,presentation,deactivated,anonymous
0,1,first,last,Göteborg,SWE,0,177,73,1996,,...,,,"Railay beach, Krabi, Thailand",None,,,1976-03-10,,0,0
1,2,first,last,stockholm,SWE,0,0,0,2000,,...,,,,,,,None,,0,0
2,3,first,last,Umeå,SWE,0,180,78,1995,,...,,,Hell,Umeå,,,1973-09-09,,0,0
3,4,first,last,Goteborg,SWE,1,165,58,2001,,...,,,,None,,,1984-07-26,,0,0
4,5,first,last,North Attleboro,USA,0,0,0,1991,,...,,,,None,,,1969-05-07,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62588,67021,first,last,Oreña,ESP,0,180,78,0,,...,,,,None,,,1991-01-05,,0,0
62589,67022,first,last,Paris,FRA,0,185,68,2016,,...,,,,None,,,1989-05-18,,0,0
62590,67023,first,last,Cody,USA,0,190,88,2001,,...,,,,None,,,1986-07-10,,0,0
62591,67024,first,last,Kraków,POL,1,0,0,0,None,...,None,None,None,None,None,None,None,None,0,0


In [7]:
df_user.columns

Index(['id', 'first_name', 'last_name', 'city', 'country', 'sex', 'height',
       'weight', 'started', 'competitions', 'occupation', 'sponsor1',
       'sponsor2', 'sponsor3', 'best_area', 'worst_area', 'guide_area',
       'interests', 'birth', 'presentation', 'deactivated', 'anonymous'],
      dtype='object')

In [8]:
#Clean up user data 
dropped_user_columns =['competitions', 'occupation', 'sponsor1',
       'sponsor2', 'sponsor3', 'best_area', 'worst_area', 'guide_area',
       'interests','presentation', 'deactivated', 'anonymous']
df_user.rename(columns={'id': 'user_id'}, inplace=True)
cleaned_up_users = df_user.drop(dropped_user_columns, axis=1)
cleaned_up_users['sex'] = cleaned_up_users['sex'].map({0:'M', 1:'F'})
cleaned_up_users

,user_id,first_name,last_name,city,country,sex,height,weight,started,birth
0,1,first,last,Göteborg,SWE,M,177,73,1996,1976-03-10
1,2,first,last,stockholm,SWE,M,0,0,2000,None
2,3,first,last,Umeå,SWE,M,180,78,1995,1973-09-09
3,4,first,last,Goteborg,SWE,F,165,58,2001,1984-07-26
4,5,first,last,North Attleboro,USA,M,0,0,1991,1969-05-07
...,...,...,...,...,...,...,...,...,...,...
62588,67021,first,last,Oreña,ESP,M,180,78,0,1991-01-05
62589,67022,first,last,Paris,FRA,M,185,68,2016,1989-05-18
62590,67023,first,last,Cody,USA,M,190,88,2001,1986-07-10
62591,67024,first,last,Kraków,POL,F,0,0,0,None


In [9]:
df_grade

,id,score,fra_routes,fra_routes_input,fra_routes_selector,fra_boulders,fra_boulders_input,fra_boulders_selector,usa_routes,usa_routes_input,usa_routes_selector,usa_boulders,usa_boulders_input,usa_boulders_selector
0,1,0,-,1,1,-,1,1,3/4,1,1,VB,1,1
1,2,0,1,0,0,1,0,0,,0,0,,0,0
2,3,0,1a,0,0,1A,0,0,,0,0,,0,0
3,4,0,1b,0,0,1B,0,0,,0,0,,0,0
4,5,0,1c,0,0,1C,0,0,,0,0,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,82,1475,9b+/9c,0,0,9B+/9C,0,0,,0,0,,0,0
79,83,1500,9c,0,0,9C,0,0,,0,0,,0,0
80,84,1525,9c/+,0,0,9C/+,0,0,,0,0,,0,0
81,85,1550,9c+,0,0,9C+,0,0,,0,0,,0,0


In [10]:
df_grade.columns

Index(['id', 'score', 'fra_routes', 'fra_routes_input', 'fra_routes_selector',
       'fra_boulders', 'fra_boulders_input', 'fra_boulders_selector',
       'usa_routes', 'usa_routes_input', 'usa_routes_selector', 'usa_boulders',
       'usa_boulders_input', 'usa_boulders_selector'],
      dtype='object')

In [11]:
#Clean up grades
df_grade.rename(columns={'id':'grade_id'}, inplace=True)
drop_fra_columns = ['score', 'fra_routes', 'fra_routes_input', 'fra_routes_selector',
       'fra_boulders', 'fra_boulders_input', 'fra_boulders_selector',
       'usa_routes', 'usa_routes_input', 'usa_routes_selector','usa_boulders_selector', 'usa_boulders_input']
usa_boulders = df_grade.drop(drop_fra_columns, axis=1)
usa_boulders

,grade_id,usa_boulders
0,1,VB
1,2,
2,3,
3,4,
4,5,
...,...,...
78,82,
79,83,
80,84,
81,85,


In [12]:
usa_boulders.usa_boulders.values

array(['VB', '', '', '', '', '', 'VB', '', '', '', 'VB', 'VB', 'VB', '',
       'VB', '', 'V0-', '', '', '', 'V0', '', 'V0', '', 'V0', '', '', '',
       'V1', 'V1', 'V1', 'V1', 'V2', 'V2', '', 'V3', '', 'V3/4', '', 'V4',
       '', 'V4/V5', 'V4/V5', 'V5', 'V5', 'V5/V6', '', 'V6', '', 'V7', '',
       'V8', '', 'V8/9', '', 'V9', '', 'V10', '', 'V11', '', 'V12', '',
       'V13', '', 'V14', '', 'V15', 'V15/16', 'V16', 'V16/17', 'V17',
       'V17/18', 'V18', 'V18/19', 'V19', 'V19/20', 'V20', '', '', '', '',
       ''], dtype=object)

In [13]:
usa_boulders_grades = usa_boulders[usa_boulders.usa_boulders.values != '']
usa_boulders_grades

,grade_id,usa_boulders
0,1,VB
6,7,VB
10,11,VB
11,12,VB
12,13,VB
14,15,VB
16,17,V0-
20,21,V0
22,23,V0
24,25,V0


In [14]:
usa_boulder_grades.drop_duplicates(inplace=True)
usa_boulder_grades.values

/var/folders/b8/qw7xf4r16w383xrhw3980l4w0000gp/T/ipykernel_11303/765187295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_boulder_grades.drop_duplicates(inplace=True)


array(['VB', 'V0-', 'V0', 'V1', 'V2', 'V3', 'V3/4', 'V4', 'V4/V5', 'V5',
       'V5/V6', 'V6', 'V7', 'V8', 'V8/9', 'V9', 'V10', 'V11', 'V12',
       'V13', 'V14', 'V15', 'V15/16', 'V16', 'V16/17', 'V17', 'V17/18',
       'V18', 'V18/19', 'V19', 'V19/20', 'V20'], dtype=object)

In [15]:
df_ascent

,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,...,country,comment,rating,description,yellow_id,climb_try,repeat,exclude_from_ranking,user_recommended,chipped
0,2,1,36,,0,3,0,545,918342000,1999,...,THA,Thailand\n,0,,255,0,0,0,0,0
1,3,1,36,,0,3,0,545,925509600,1999,...,,\n,0,,0,0,0,0,0,0
2,4,1,36,,0,3,0,545,933026400,1999,...,SWE,\n,0,,255,0,0,0,0,0
3,5,1,36,,0,3,0,545,933026400,1999,...,SWE,\n,0,,255,0,0,0,0,0
4,6,1,36,,0,3,0,545,933458400,1999,...,,\n,0,,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4111872,4910370,59627,46,,0,1,0,650,1505167200,2017,...,POL,"miała być rozgrzewka, a w pełnym słońcu był ca...",2,,255,0,0,0,0,0
4111873,4910371,44075,49,,0,1,0,700,1505080800,2017,...,POL,,0,,255,0,0,0,0,0
4111874,4910376,19098,51,,0,3,0,895,1504994400,2017,...,FRA,"Sobada, al igual que el 7b, 7c i l'altre 7c de...",0,,255,0,0,0,0,0
4111875,4910379,19098,57,,0,3,0,1045,1505080800,2017,...,FRA,"El 7a és guarrillu, però la part de dalt és bo...",2,,255,0,0,0,0,0


In [16]:
df_ascent.columns

Index(['id', 'user_id', 'grade_id', 'notes', 'raw_notes', 'method_id',
       'climb_type', 'total_score', 'date', 'year', 'last_year', 'rec_date',
       'project_ascent_date', 'name', 'crag_id', 'crag', 'sector_id', 'sector',
       'country', 'comment', 'rating', 'description', 'yellow_id', 'climb_try',
       'repeat', 'exclude_from_ranking', 'user_recommended', 'chipped'],
      dtype='object')

In [17]:
dropped_columns_ascent = ['id', 'notes', 'raw_notes', 'total_score', 'comment', 'description', 'yellow_id', 'climb_try', 'repeat', 'exclude_from_ranking','user_recommended','chipped','rating','project_ascent_date','last_year','climb_type','rec_date','date']
cleaned_up_ascent = df_ascent.drop(dropped_columns_ascent, axis=1)
cleaned_up_ascent

,user_id,grade_id,method_id,year,name,crag_id,crag,sector_id,sector,country
0,1,36,3,1999,The King And I,16596,Railay,61,Dum's kitchen,THA
1,1,36,3,1999,vet ej,0,Nya berg - segl.,0,,
2,1,36,3,1999,Mr Big,209,Sjöända,371,Huvudväggen,SWE
3,1,36,3,1999,Tak ska du ha,209,Sjöända,371,Huvudväggen,SWE
4,1,36,3,1999,Korpen flyger,0,"Rankulla, Vetlanda",0,,
...,...,...,...,...,...,...,...,...,...,...
4111872,59627,46,1,2017,Prawy Meningitis,28984,Dolina Bedkowska,102,Dupa Slonia,POL
4111873,44075,49,1,2017,"Przyczajony Kucyk, Ukryty Smok",29014,Dolina Kluczwody,1623,Jaskinia Mamutowa,POL
4111874,19098,51,3,2017,Le cri du papillon,867,Mont Dauphin,2244,El Pouding,FRA
4111875,19098,57,3,2017,Central street,28033,Rue des masques,413,Le Canyon,FRA


In [18]:
combined_data = cleaned_up_users.merge(cleaned_up_ascent,on='user_id').merge(usa_boulders, on='grade_id')
combined_data

,user_id,first_name,last_name,city,country_x,sex,height,weight,started,birth,grade_id,method_id,year,name,crag_id,crag,sector_id,sector,country_y,usa_boulders
0,1,first,last,Göteborg,SWE,M,177,73,1996,1976-03-10,36,3,1999,The King And I,16596,Railay,61,Dum's kitchen,THA,V3
1,1,first,last,Göteborg,SWE,M,177,73,1996,1976-03-10,36,3,1999,vet ej,0,Nya berg - segl.,0,,,V3
2,1,first,last,Göteborg,SWE,M,177,73,1996,1976-03-10,36,3,1999,Mr Big,209,Sjöända,371,Huvudväggen,SWE,V3
3,1,first,last,Göteborg,SWE,M,177,73,1996,1976-03-10,36,3,1999,Tak ska du ha,209,Sjöända,371,Huvudväggen,SWE,V3
4,1,first,last,Göteborg,SWE,M,177,73,1996,1976-03-10,36,3,1999,Korpen flyger,0,"Rankulla, Vetlanda",0,,,V3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4110928,1476,first,last,Brno,CZE,M,185,68,1999,1993-02-05,83,1,2017,Silence,29245,Flatanger,5982,Hanshallaren,NOR,
4110929,37502,first,last,Kent,USA,M,172,53,2009,1999-10-05,83,1,2012,The Game,27654,Boulder canyon,0,,USA,
4110930,46207,first,last,Saint Louis,USA,M,175,58,2010,1997-08-27,83,4,2013,Swag,0,Holy boulders,0,Swag,USA,
4110931,48467,first,last,London,GBR,M,182,78,2012,None,83,3,2017,DUMMY,2452,Kalymnos,0,DUMMY,GRC,


In [ ]:
com_dups = combined_data.drop_duplicates(subset=None, keep='first', inplace=False)
com_dups

In [ ]:
com_dups.rename(columns={'country_x':'user_country'}, inplace=True)
com_dups.rename(columns={'country_y':'boulder_country'}, inplace=True)
com_dups.rename(columns={'usa_boulders':'usa_boulder_scale'}, inplace=True)

In [ ]:
dropped_com_dups = ['first_name', 'last_name']
com_dups.drop(dropped_com_dups, axis=1, inplace=True)
com_dups

In [ ]:
gender_v_grade = com_dups[['sex','usa_boulder_scale']]
gender_v_grade

In [ ]:
male_grade = gender_v_grade.loc[gender_v_grade['sex'] == 'M']
male_grade

In [ ]:
female_grade = gender_v_grade.loc[gender_v_grade['sex'] == 'F']
female_grade